# Example of how to use TopologicPy with Neo4J Graph Database

### Step 1: Import the needed libraries

In [1]:
# The following two lines are not needed on your computer
import sys
sys.path.append(r"C:\Users\sarwj\OneDrive - Cardiff University\Documents\GitHub\topologicpy\src")


In [2]:
from topologicpy.Neo4j import Neo4j
from topologicpy.Topology import Topology
from topologicpy.Graph import Graph
from topologicpy.CellComplex import CellComplex
from topologicpy.Dictionary import Dictionary
from getpass import getpass


### Step 2: Enter your credentials. Make sure you have your Neo4J DBMS running and that you know the URI, username, and password

In [ ]:
print("Enter your credentials!")
default_url = "bolt://localhost:7687"
url = input(f"Enter the Neo4j DBMS URL (default: {default_url}): ") or default_url

default_username = "neo4j"
# Input username
# Prompt the user for the username, allowing for a default
username = input(f"Enter your Neo4j username (default: {default_username}): ") or default_username

# Input password (hidden)
password = getpass("Enter your Neo4j password: ")
print("Done. Move to next step")

### Step 3: Get the Neo4J Graph (in reality this is the driver, but we can think of it as the neo4j graph)

In [ ]:
n_graph = Neo4j.ByParameters(url, username, password)
print(n_graph)


### Step 4: Reset the graph (Delete all existing nodes and edges. Be careful with this step if you need to keep your graph)

In [ ]:
n_graph = Neo4j.Reset(n_graph)
print(n_graph)

### Step 5: Create a Topologic Graph

In [ ]:
cc = CellComplex.Prism(width=30, length=20, height=20, uSides=3)
cells = Topology.Cells(cc)
for i, cell in enumerate(cells):
    d = Dictionary.ByKeyValue("label", "Cell_"+str(i+1))
    cell = Topology.SetDictionary(cell, d)
faces = Topology.Faces(cc)
for i, face in enumerate(faces):
    d = Dictionary.ByKeyValue("label", "Face_"+str(i+1))
    face = Topology.SetDictionary(face, d)

t_graph = Graph.ByTopology(cc, direct=False, toExteriorTopologies=True, viaSharedTopologies=True)
vertices = Graph.Vertices(t_graph)
edges = Graph.Edges(t_graph)
print("This graph has", len(vertices), "vertices and", len(edges), "edges.")
print("Vertex Dictionaries:")
for v in vertices:
    d = Topology.Dictionary(v)
    print(" ", Dictionary.Keys(d), Dictionary.Values(d))
print("Edge Dictionaries:")
for e in edges:
    d = Topology.Dictionary(e)
    print(" ", Dictionary.Keys(d), Dictionary.Values(d))


### Step 6: Convert the Topologic Graph to a Neo4J graph.

In [ ]:
n_graph = Neo4j.ByGraph(n_graph,
                           t_graph,
                           vertexLabelKey="label",
                           defaultVertexLabel="Untitled",
                           vertexCategoryKey="category",
                           defaultVertexCategory=None,
                           edgeLabelKey="relationship",
                           defaultEdgeLabel="CONNECTED_TO",
                           edgeCategoryKey= "category",
                           defaultEdgeCategory=None,
                           mantissa=6,
                           tolerance = 0.0001)
print(n_graph)


### Step 7: Convert the Neo4J graph to a Topologic Graph

In [ ]:
t_graph = Neo4j.ExportToGraph(n_graph)
Graph.Show(t_graph, vertexGroupKey="category", vertexGroups=[0,1,3], vertexSize=12, vertexLabelKey="label", edgeLabelKey="relationship", colorScale="thermal_r")
vertices = Graph.Vertices(t_graph)
for v in vertices:
    d = Topology.Dictionary(v)
    print(Dictionary.Keys(d), Dictionary.Values(d))

### Step 8: Get ONLY the vertices and edges that match the cypher string

In [ ]:
cypher="MATCH (a)-[r:To_Exterior_Topologies]->(b) RETURN a,r,b"
t_graph = Neo4j.ExportToGraph(n_graph, cypher=cypher)
Graph.Show(t_graph, vertexGroupKey="category", vertexGroups=[0,1,3], vertexSize=12, vertexLabelKey="label", edgeLabelKey="relationship")
vertices = Graph.Vertices(t_graph)
for v in vertices:
    d = Topology.Dictionary(v)
    print(Dictionary.Keys(d), Dictionary.Values(d))

In [ ]:
cypher="MATCH (a)-[r:Via_Shared_Topologies]->(b) RETURN a,r,b"
t_graph = Neo4j.ExportToGraph(n_graph, cypher=cypher)
Graph.Show(t_graph, vertexGroupKey="category", vertexGroups=[0,1,2,3], vertexSize=12, vertexLabelKey="label", edgeLabelKey="relationship", colorScale="thermal_r")
vertices = Graph.Vertices(t_graph)
for v in vertices:
    d = Topology.Dictionary(v)
    print(Dictionary.Keys(d), Dictionary.Values(d))

### Step 9: Try another cypher query (only nodes, no edges)

In [ ]:
cypher = "MATCH (n) WHERE ANY(label IN labels(n) WHERE label STARTS WITH 'Cell') RETURN n"
t_graph = Neo4j.ExportToGraph(n_graph, cypher=cypher)
vertices = Graph.Vertices(t_graph)
for v in vertices:
    d = Topology.Dictionary(v)
    print(Dictionary.Keys(d), Dictionary.Values(d))
Graph.Show(t_graph, vertexGroupKey="category", vertexGroups=[0,1,3], vertexSize=12, vertexLabelKey="label", edgeLabelKey="relationship", , colorScale="thermal_r")
